# 分割训练集和验证集
#### 注意筛选gt2d<0的帧

In [1]:
import math, json, sys, os
from os.path import join
import json
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm import tqdm
from multiprocessing.dummy import Pool

In [2]:
data_dir = '/home/windward/gcy/Golf/proj/data/SURREAL/summary'
image_dir = os.path.join(data_dir, 'image')
label_dir = os.path.join(data_dir, 'labels')
split_data_dir = '/home/windward/gcy/Golf/proj/data'

In [3]:
label_files = []
label_files_filter = []

def run(label_file):
    label_path = join(label_dir, label_file)
    with open(label_path) as f:
        data = json.load(f)
    gt2d = np.array(data['joints2D']).reshape((2, 24)).T
    min_pt = np.min(gt2d, axis=0)    # 最小的x和y
    max_pt = np.max(gt2d, axis=0)    # 最大的x和y
    if (min_pt[0] < 0 or max_pt[0] > 320 or min_pt[1] < 0 or max_pt[1] > 240):
        return label_file
    else:
        return 0
    
label_files = sorted(os.listdir(label_dir))
pool = Pool(20)    #创建n个容量的线程池并发执行
for f in tqdm(pool.imap(run, label_files)):
    if f != 0:
        label_files_filter.append(f)
    
pool.close()
pool.join()

6396439it [45:44, 2330.24it/s]


In [4]:
len(label_files_filter)

266033


In [5]:
# 将需要滤除的名字存入文件
pd_t = pd.DataFrame({'name_filter': label_files_filter})['name_filter'].map(lambda x: x[:-5]).to_csv(
    os.path.join(split_data_dir, 'names_filter.csv'), header='name_filter', index=False)

In [6]:
names_filter = pd.read_csv(os.path.join(split_data_dir, 'names_filter.csv'))['name_filter']
labels_all = set(os.listdir(label_dir))
labels_filter = set(list(names_filter.map(lambda x: x + '.json')))
labels_use = sorted(list(labels_all - labels_filter))
images_use = [i[:-4] + 'jpg' for i in labels_use]

In [7]:
len(images_use)

6130406

In [14]:
image_name_train = images_use[:-20000]
image_name_valid = images_use[-20000:]
# image_name_train, image_name_valid = train_test_split(
#     images_names, test_size=0.1, random_state=1, shuffle=False)
pd.DataFrame({'name': image_name_train}).to_csv(os.path.join(
    split_data_dir, 'surreal_train_names.csv'), index=False)
pd.DataFrame({'name': image_name_valid}).to_csv(os.path.join(
    split_data_dir, 'surreal_valid_names.csv'), index=False)